<a href="https://colab.research.google.com/github/manuelboi/MLsec_project/blob/main/MLsec_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Models:
1. Fixing Data Augmentation to Improve Adversarial Robustness
2. Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness?
3. MMA Training: Direct Input Space Margin Maximization through Adversarial Training

# General imports

In [ ]:
%%capture

# Various
import matplotlib.pyplot as plt
import time
import importlib.util
import re

# Pytorch
import torch
from torch.utils.data import DataLoader, Subset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

# RobustBench
!pip install git+https://github.com/manuelboi/robustbench
from robustbench.utils import load_model
from robustbench.eval import benchmark
from robustbench.data import load_cifar10
from robustbench.model_zoo.enums import ThreatModel

# Smoothing
!git clone https://github.com/matteoturnu/smoothing.git
!conda create -n smoothing
!conda activate smoothing
!conda install pytorch torchvision cudatoolkit=10.0 -c pytorch
!conda install scipy pandas statsmodels matplotlib seaborn
!pip install setGPU
from smoothing.code.core import Smooth
from smoothing.code.train import train, test
from smoothing.code.train_utils import AverageMeter, accuracy

# Setup

In [ ]:
%%capture

# Set labels
labels_dct = {0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"}

# Use GPU if available, otherwise use CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Computing device used: ", device)

# Preparing trainset and testset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Set trainset and testset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Define the number of samples you want to take from the whole dataset
num_samples = 1000

# Create a subset of the dataset with the chosen number of samples
indices = list(range(num_samples))
train_subset = Subset(trainset, indices)
test_subset = Subset(testset, indices)

# Preparing trainloader and testloader
batch_size = 64
trainloader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size, shuffle=False, num_workers=2)

# Load test samples for predict and certify
n_test_samples_benchmark = 100
x_test, y_test = load_cifar10(n_test_samples_benchmark)
x_test, y_test = x_test.to(device), y_test.to(device)

# Setting various parameters
epochs = 3 # used for training
sigma = 0.25 # gaussian noise standard deviation
n_examples = 20 # to be perturbed by AutoAttack
eps_L2 = 0.5 # epsilon of the perturbancy for L2 norm
eps_Linf = 8/255 # epsilon of the perturbancy with Linf norm
version = 'custom'
attacks_to_run=['apgd-ce', 'apgd-dlr'] # Attacks to run on the models

# Functions

Function to compact donwload and configuration of the model

In [ ]:
def load_configure_model(name, dataset, threat_model, device):
  # Download model from robustbench
  model = load_model(model_name=name, dataset=dataset, threat_model=threat_model)

  # Set model to the chosen device
  model = model.to(device)

  # Set model in eval mode
  model.eval()

  return model

Function for smooth model training

In [ ]:
def train_model(trainloader, testloader, model, epochs, sigma, device):
  criterion = CrossEntropyLoss().to(device)
  optimizer = SGD(model.parameters())
  scheduler = StepLR(optimizer, step_size=30)

  for epoch in range(epochs):
    scheduler.step(epoch)
    before = time.time()
    train_loss, train_acc = train(trainloader, model, criterion, optimizer, epoch, sigma, device)
    test_loss, test_acc = test(testloader, model, criterion, epoch, sigma, device)
    after = time.time()

  return model

Function for measuring performances on smooth model

In [ ]:
def certify(model, sigma, x_text, y_test, L):
  n_classes = 10
  alpha = 1 # (1 - alpha) is the confidence level (in this case is 0%)
  n0 = 10 # number of samples for selection
  n = 20 # number of samples for estimation (certify) (too few samples but computation time is strongly affected with more)

  smooth_model = Smooth(model, n_classes, sigma)

  top_classes = list()
  radiuses = list()
  for x, y in zip(x_test, y_test):
    top_class = smooth_model.predict(x, n0, alpha, batch_size=n0, device=device,)
    top_class, radius = smooth_model.certify(x, n0, n, alpha, batch_size=n0, device=device, L=L)
    top_classes.append(top_class)
    radiuses.append(radius)

  top_classes = torch.tensor(top_classes, dtype=torch.float64).to(device)
  accuracy = torch.mean(top_classes == y_test, dtype=torch.float64)
  print("Radiuses list: ", radiuses)
  radius = torch.mean(torch.Tensor(radiuses), dtype=torch.float64)


  print("Top classes: ", top_classes)
  print("Y classes: ", y_test)
  print(f"Average radious found by certify: {float(radius.item())}")
  percentage = float(accuracy.item()) * 100
  print(f"Accuracy found by certify function: {percentage:.2f}%")

Function to compact test of the model with certification and AutoAttack

In [ ]:
def test_model(model, sigma, x_test, y_test, norm, threat_model, n_examples, eps, batch_size, device, version, attacks_to_run):
  # Model prediction and certification on smoothed samples
  certify(model, sigma, x_test, y_test, norm)

  # AutoAttack on given model
  benchmark(model,
            threat_model=threat_model,
            n_examples=n_examples,
            eps=eps,
            batch_size=batch_size,
            device=device,
            version=version,
            attacks_to_run=attacks_to_run)

# Fixing Data Augmentation to Improve Adversarial Robustness (WideResNet-70-16)

L2

In [ ]:
# Model loading
model_1_L2 = load_configure_model(name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='L2', device=device)

# Test on stock model
test_model(model_1_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_1_L2 = train_model(trainloader, testloader, model_1_L2, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_1_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

Linf

In [ ]:
# Model loading
model_1_Linf = load_configure_model(name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='Linf', device=device)

# Test on stock model
test_model(model_1_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_1_Linf = train_model(trainloader, testloader, model_1_Linf, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_1_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

# Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness? (ResNet-18)

L2

In [ ]:
# Model loading
model_2_L2 = load_configure_model(name='Sehwag2021Proxy_R18', dataset='cifar10', threat_model='L2', device=device)

# Test on stock model
test_model(model_2_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_2_L2 = train_model(trainloader, testloader, model_2_L2, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_2_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

Linf

In [ ]:
# Model loading
model_2_Linf = load_configure_model(name='Sehwag2021Proxy_R18', dataset='cifar10', threat_model='L2', device=device)

# Test on stock model
test_model(model_2_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_2_Linf = train_model(trainloader, testloader, model_2_Linf, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_2_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

# MMA Training: Direct Input Space Margin Maximization through Adversarial Training (WideResNet-28-4)

L2

In [ ]:
# Model loading
model_3_L2 = load_configure_model(name='Ding2020MMA', dataset='cifar10', threat_model='L2', device=device)

# Test on stock model
test_model(model_3_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_3_L2 = train_model(trainloader, testloader, model_3_L2, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_3_L2, sigma, x_test, y_test, 'L2', ThreatModel.L2, n_examples, eps_L2, batch_size, device, version, attacks_to_run)

Linf

In [ ]:
# Model loading
model_3_Linf = load_configure_model(name='Ding2020MMA', dataset='cifar10', threat_model='Linf', device=device)

# Test on stock model
test_model(model_3_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)

# Smoothed model training
smoothed_model_3_Linf = train_model(trainloader, testloader, model_3_Linf, epochs, sigma, device)

# Test on smoothed model
test_model(smoothed_model_3_Linf, sigma, x_test, y_test, 'Linf', ThreatModel.Linf, n_examples, eps_Linf, batch_size, device, version, attacks_to_run)